## 제주 빅데이터 경진대회
#### 이민혁
### Problem : 2020.04, 2020.07 기간 내 지역, 업종 별 월간 총 사용 금액 예측 
- 2019.01 ~ 2020.03 기간 내 카드 데이터, 201901-202003.csv (2.07 GB)
- 2020.04 기간 내 카드 데이터 (7/28 공개), 202004.csv (116 MB)
- 제출 양식, submission.csv (64 KB)


### 가장 최근에 나온 CatBoostRegression 사용
#### EDA(x), 베이스 라인 전처리 그대로, CatBoost 파라미터 튜닝없는 기본 값으로 예측

## 1. 라이브러리, 데이터 불러오기

In [1]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
from ## 1. 라이브러리, 데이터 불러오기sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('201901-202003.csv')
data.head()

REG_YYMM CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM HOM_SIDO_NM HOM_CCG_NM  AGE  \
0    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  20s   
1    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  30s   
2    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  30s   
3    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  40s   
4    201901           강원         강릉시  건강보조식품 소매업          강원        강릉시  40s   

   SEX_CTGO_CD  FLC  CSTMR_CNT      AMT  CNT  
0            1    1          4   311200    4  
1            1    2          7  1374500    8  
2            2    2          6   818700    6  
3            1    3          4  1717000    5  
4            1    4          3  1047300    3

**데이터 설명**

REG__YYMM = 년월

CARD_SIDO_NM = 카드이용지역_시도 (가맹점 주소 기준)

CARD_CCG_NM = 카드이용지역_시군구 (가맹점 주소 기준)

STD_CLSS_NM = 업종명

HOM_SIDO_NM = 거주지역_시도 (고객 집주소 기준)

HOM_CCG_NM = 거주지역_시군구 (고객 집주소 기준)

AGE = 연령대

SEX_CTGO_CD = 성별 (1: 남성, 2: 여성)

FLC = 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)

CSTMR_CNT = 이용고객수 (명)

AMT = 이용금액 (원)

CNT = 이용건수 (건)

## 2. 데이터 전처리

#### 1) 날짜 처리

In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return in

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

#### 2) NULL값 처리(필요 없는 컬럼 삭제)

In [4]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

#### 3) 라벨인코딩

In [5]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

#### 4) 2020년 3월 데이터를 예측데이터로 설정

In [6]:
df_sample = df_num.copy()
test = df_num[(df_num['year'] == 2020) & (df_sample['month'] == 3)]
df_sample = df_num.drop(test.index)

In [7]:
test

CARD_SIDO_NM  STD_CLSS_NM  HOM_SIDO_NM  AGE  SEX_CTGO_CD  FLC  year  \
7                   0            0            0    1            1    1  2020   
43                  0            0            0    2            1    2  2020   
59                  0            0            0    2            2    2  2020   
67                  0            0            0    3            1    2  2020   
82                  0            0            0    3            1    3  2020   
...               ...          ...          ...  ...          ...  ...   ...   
1057283            16           40           16    3            1    4  2020   
1057328            16           40           16    4            1    4  2020   
1057343            16           40           16    4            2    4  2020   
1057358            16           40           16    5            1    5  2020   
1057373            16           40           16    5            2    5  2020   

         month  CSTMR_CNT      AMT  CNT  
7            3          3   427510    2  
43           3         38  5622890   37  
59           3         30  2296125   38  
67           3          3   257000    4  
82           3         51  6490800   48  
...        ...        ...      ...  ...  
1057283      3          9   385700   10  
1057328      3         10   790350   19  
1057343      3          4    99000    6  
1057358      3          6   228800    6  
1057373      3          3    51500    3  

[54646 rows x 11 columns]

## 4. 변수 선택 및 모델 구축

In [31]:
from sklearn.model_selection import train_test_split , GridSearchCV

train_features = df_num.drop(['AMT','CSTMR_CNT','CNT'], axis=1)
train_target = np.log1p(df_num['AMT'])

In [26]:
pool=Pool(data=train_features,label=train_target)
print (train_features.shape)

(1057394, 8)


In [27]:
model = CatBoostRegressor()

**CatBoostRegressor : 기본 파라미터**

In [28]:
test_features = test.drop(['AMT','CSTMR_CNT','CNT'], axis=1)
test_target = np.log1p(test['AMT'])
eval_set=(test_features,test_target)
model.fit(train_features,train_target,use_best_model=True,eval_set=eval_set)

Learning rate set to 0.171818
0:	learn: 2.4681472	test: 2.5172747	best: 2.5172747 (0)	total: 72.7ms	remaining: 1m 12s
1:	learn: 2.4397326	test: 2.4893863	best: 2.4893863 (1)	total: 137ms	remaining: 1m 8s
2:	learn: 2.4182896	test: 2.4678413	best: 2.4678413 (2)	total: 200ms	remaining: 1m 6s
3:	learn: 2.4046683	test: 2.4567370	best: 2.4567370 (3)	total: 257ms	remaining: 1m 4s
4:	learn: 2.3841910	test: 2.4363610	best: 2.4363610 (4)	total: 315ms	remaining: 1m 2s
5:	learn: 2.3675721	test: 2.4190087	best: 2.4190087 (5)	total: 379ms	remaining: 1m 2s
6:	learn: 2.3581124	test: 2.4117849	best: 2.4117849 (6)	total: 434ms	remaining: 1m 1s
7:	learn: 2.3474295	test: 2.4016177	best: 2.4016177 (7)	total: 493ms	remaining: 1m 1s
8:	learn: 2.3252945	test: 2.3793865	best: 2.3793865 (8)	total: 555ms	remaining: 1m 1s
9:	learn: 2.3172440	test: 2.3724734	best: 2.3724734 (9)	total: 621ms	remaining: 1m 1s
10:	learn: 2.3097789	test: 2.3662834	best: 2.3662834 (10)	total: 684ms	remaining: 1m 1s
11:	learn: 2.2923589

96:	learn: 1.8738138	test: 1.9415679	best: 1.9415679 (96)	total: 5.96s	remaining: 55.5s
97:	learn: 1.8718932	test: 1.9409028	best: 1.9409028 (97)	total: 6.02s	remaining: 55.4s
98:	learn: 1.8702446	test: 1.9389680	best: 1.9389680 (98)	total: 6.07s	remaining: 55.3s
99:	learn: 1.8670650	test: 1.9351388	best: 1.9351388 (99)	total: 6.13s	remaining: 55.2s
100:	learn: 1.8656967	test: 1.9337680	best: 1.9337680 (100)	total: 6.19s	remaining: 55.1s
101:	learn: 1.8647878	test: 1.9318338	best: 1.9318338 (101)	total: 6.25s	remaining: 55s
102:	learn: 1.8626159	test: 1.9299299	best: 1.9299299 (102)	total: 6.31s	remaining: 54.9s
103:	learn: 1.8609776	test: 1.9288488	best: 1.9288488 (103)	total: 6.37s	remaining: 54.9s
104:	learn: 1.8589094	test: 1.9268142	best: 1.9268142 (104)	total: 6.43s	remaining: 54.8s
105:	learn: 1.8579505	test: 1.9256360	best: 1.9256360 (105)	total: 6.48s	remaining: 54.7s
106:	learn: 1.8536782	test: 1.9212572	best: 1.9212572 (106)	total: 6.54s	remaining: 54.6s
107:	learn: 1.852423

189:	learn: 1.6845658	test: 1.7606455	best: 1.7606455 (189)	total: 11.5s	remaining: 49.2s
190:	learn: 1.6831007	test: 1.7591071	best: 1.7591071 (190)	total: 11.6s	remaining: 49.1s
191:	learn: 1.6771081	test: 1.7524161	best: 1.7524161 (191)	total: 11.7s	remaining: 49s
192:	learn: 1.6753676	test: 1.7513674	best: 1.7513674 (192)	total: 11.7s	remaining: 49s
193:	learn: 1.6747275	test: 1.7507896	best: 1.7507896 (193)	total: 11.8s	remaining: 48.9s
194:	learn: 1.6738995	test: 1.7506052	best: 1.7506052 (194)	total: 11.8s	remaining: 48.8s
195:	learn: 1.6727019	test: 1.7494814	best: 1.7494814 (195)	total: 11.9s	remaining: 48.8s
196:	learn: 1.6683129	test: 1.7442012	best: 1.7442012 (196)	total: 12s	remaining: 48.7s
197:	learn: 1.6675161	test: 1.7434898	best: 1.7434898 (197)	total: 12s	remaining: 48.7s
198:	learn: 1.6666252	test: 1.7425283	best: 1.7425283 (198)	total: 12.1s	remaining: 48.6s
199:	learn: 1.6634794	test: 1.7389399	best: 1.7389399 (199)	total: 12.1s	remaining: 48.5s
200:	learn: 1.6615

283:	learn: 1.5540429	test: 1.6244943	best: 1.6244943 (283)	total: 17.2s	remaining: 43.3s
284:	learn: 1.5527736	test: 1.6234612	best: 1.6234612 (284)	total: 17.2s	remaining: 43.3s
285:	learn: 1.5518226	test: 1.6222841	best: 1.6222841 (285)	total: 17.3s	remaining: 43.2s
286:	learn: 1.5500562	test: 1.6202758	best: 1.6202758 (286)	total: 17.4s	remaining: 43.1s
287:	learn: 1.5474285	test: 1.6173953	best: 1.6173953 (287)	total: 17.4s	remaining: 43.1s
288:	learn: 1.5467875	test: 1.6170780	best: 1.6170780 (288)	total: 17.5s	remaining: 43s
289:	learn: 1.5460969	test: 1.6165820	best: 1.6165820 (289)	total: 17.5s	remaining: 43s
290:	learn: 1.5447316	test: 1.6155665	best: 1.6155665 (290)	total: 17.6s	remaining: 42.9s
291:	learn: 1.5442753	test: 1.6152163	best: 1.6152163 (291)	total: 17.7s	remaining: 42.8s
292:	learn: 1.5438790	test: 1.6148985	best: 1.6148985 (292)	total: 17.7s	remaining: 42.8s
293:	learn: 1.5429812	test: 1.6141759	best: 1.6141759 (293)	total: 17.8s	remaining: 42.7s
294:	learn: 1.

377:	learn: 1.4587101	test: 1.5293227	best: 1.5293227 (377)	total: 22.9s	remaining: 37.6s
378:	learn: 1.4580671	test: 1.5289353	best: 1.5289353 (378)	total: 22.9s	remaining: 37.5s
379:	learn: 1.4570681	test: 1.5281696	best: 1.5281696 (379)	total: 23s	remaining: 37.5s
380:	learn: 1.4566018	test: 1.5281366	best: 1.5281366 (380)	total: 23s	remaining: 37.4s
381:	learn: 1.4562858	test: 1.5280154	best: 1.5280154 (381)	total: 23.1s	remaining: 37.4s
382:	learn: 1.4556922	test: 1.5276290	best: 1.5276290 (382)	total: 23.2s	remaining: 37.3s
383:	learn: 1.4552223	test: 1.5272458	best: 1.5272458 (383)	total: 23.2s	remaining: 37.2s
384:	learn: 1.4538986	test: 1.5261011	best: 1.5261011 (384)	total: 23.3s	remaining: 37.2s
385:	learn: 1.4526308	test: 1.5243931	best: 1.5243931 (385)	total: 23.3s	remaining: 37.1s
386:	learn: 1.4523505	test: 1.5242622	best: 1.5242622 (386)	total: 23.4s	remaining: 37.1s
387:	learn: 1.4496423	test: 1.5212528	best: 1.5212528 (387)	total: 23.5s	remaining: 37s
388:	learn: 1.44

...

565:	learn: 1.3356126	test: 1.4063196	best: 1.4063196 (565)	total: 34.2s	remaining: 26.2s
566:	learn: 1.3353616	test: 1.4063528	best: 1.4063196 (565)	total: 34.2s	remaining: 26.2s
567:	learn: 1.3349095	test: 1.4058576	best: 1.4058576 (567)	total: 34.3s	remaining: 26.1s
568:	learn: 1.3339446	test: 1.4041386	best: 1.4041386 (568)	total: 34.4s	remaining: 26s
569:	learn: 1.3317366	test: 1.4012685	best: 1.4012685 (569)	total: 34.4s	remaining: 26s
570:	learn: 1.3310284	test: 1.4005822	best: 1.4005822 (570)	total: 34.5s	remaining: 25.9s
571:	learn: 1.3307660	test: 1.4003081	best: 1.4003081 (571)	total: 34.5s	remaining: 25.9s
572:	learn: 1.3305707	test: 1.4002440	best: 1.4002440 (572)	total: 34.6s	remaining: 25.8s
573:	learn: 1.3304112	test: 1.3998256	best: 1.3998256 (573)	total: 34.7s	remaining: 25.7s
574:	learn: 1.3302710	test: 1.3997189	best: 1.3997189 (574)	total: 34.7s	remaining: 25.7s
575:	learn: 1.3300008	test: 1.3995827	best: 1.3995827 (575)	total: 34.8s	remaining: 25.6s
576:	learn: 1.

657:	learn: 1.2876175	test: 1.3568943	best: 1.3568943 (657)	total: 39.8s	remaining: 20.7s
658:	learn: 1.2873531	test: 1.3565521	best: 1.3565521 (658)	total: 39.8s	remaining: 20.6s
659:	learn: 1.2869814	test: 1.3563650	best: 1.3563650 (659)	total: 39.9s	remaining: 20.5s
660:	learn: 1.2865509	test: 1.3559291	best: 1.3559291 (660)	total: 39.9s	remaining: 20.5s
661:	learn: 1.2855552	test: 1.3548920	best: 1.3548920 (661)	total: 40s	remaining: 20.4s
662:	learn: 1.2854477	test: 1.3548798	best: 1.3548798 (662)	total: 40.1s	remaining: 20.4s
663:	learn: 1.2851695	test: 1.3545682	best: 1.3545682 (663)	total: 40.1s	remaining: 20.3s
664:	learn: 1.2844296	test: 1.3532928	best: 1.3532928 (664)	total: 40.2s	remaining: 20.2s
665:	learn: 1.2841440	test: 1.3531803	best: 1.3531803 (665)	total: 40.2s	remaining: 20.2s
666:	learn: 1.2839033	test: 1.3529873	best: 1.3529873 (666)	total: 40.3s	remaining: 20.1s
667:	learn: 1.2837828	test: 1.3528457	best: 1.3528457 (667)	total: 40.3s	remaining: 20.1s
668:	learn: 

750:	learn: 1.2430034	test: 1.3089289	best: 1.3089289 (750)	total: 45.4s	remaining: 15s
751:	learn: 1.2427110	test: 1.3088257	best: 1.3088257 (751)	total: 45.4s	remaining: 15s
752:	learn: 1.2425965	test: 1.3087834	best: 1.3087834 (752)	total: 45.5s	remaining: 14.9s
753:	learn: 1.2423729	test: 1.3086360	best: 1.3086360 (753)	total: 45.6s	remaining: 14.9s
754:	learn: 1.2421696	test: 1.3085696	best: 1.3085696 (754)	total: 45.6s	remaining: 14.8s
755:	learn: 1.2420035	test: 1.3085699	best: 1.3085696 (754)	total: 45.7s	remaining: 14.7s
756:	learn: 1.2414522	test: 1.3079759	best: 1.3079759 (756)	total: 45.7s	remaining: 14.7s
757:	learn: 1.2410772	test: 1.3075880	best: 1.3075880 (757)	total: 45.8s	remaining: 14.6s
758:	learn: 1.2409857	test: 1.3075033	best: 1.3075033 (758)	total: 45.9s	remaining: 14.6s
759:	learn: 1.2405760	test: 1.3073094	best: 1.3073094 (759)	total: 45.9s	remaining: 14.5s
760:	learn: 1.2403707	test: 1.3073519	best: 1.3073094 (759)	total: 46s	remaining: 14.4s
761:	learn: 1.24

842:	learn: 1.2087005	test: 1.2756290	best: 1.2756290 (842)	total: 51s	remaining: 9.49s
843:	learn: 1.2085946	test: 1.2755347	best: 1.2755347 (843)	total: 51s	remaining: 9.43s
844:	learn: 1.2083618	test: 1.2753703	best: 1.2753703 (844)	total: 51.1s	remaining: 9.37s
845:	learn: 1.2079764	test: 1.2753331	best: 1.2753331 (845)	total: 51.1s	remaining: 9.31s
846:	learn: 1.2077219	test: 1.2752291	best: 1.2752291 (846)	total: 51.2s	remaining: 9.25s
847:	learn: 1.2071391	test: 1.2748407	best: 1.2748407 (847)	total: 51.3s	remaining: 9.19s
848:	learn: 1.2070366	test: 1.2747451	best: 1.2747451 (848)	total: 51.3s	remaining: 9.13s
849:	learn: 1.2069138	test: 1.2746508	best: 1.2746508 (849)	total: 51.4s	remaining: 9.07s
850:	learn: 1.2065121	test: 1.2742060	best: 1.2742060 (850)	total: 51.4s	remaining: 9.01s
851:	learn: 1.2063749	test: 1.2738652	best: 1.2738652 (851)	total: 51.5s	remaining: 8.95s
852:	learn: 1.2061818	test: 1.2735041	best: 1.2735041 (852)	total: 51.6s	remaining: 8.88s
853:	learn: 1.

935:	learn: 1.1787659	test: 1.2469525	best: 1.2469525 (935)	total: 56.6s	remaining: 3.87s
936:	learn: 1.1786873	test: 1.2467484	best: 1.2467484 (936)	total: 56.7s	remaining: 3.81s
937:	learn: 1.1783239	test: 1.2465209	best: 1.2465209 (937)	total: 56.8s	remaining: 3.75s
938:	learn: 1.1780857	test: 1.2462816	best: 1.2462816 (938)	total: 56.8s	remaining: 3.69s
939:	learn: 1.1779770	test: 1.2462819	best: 1.2462816 (938)	total: 56.9s	remaining: 3.63s
940:	learn: 1.1777548	test: 1.2460940	best: 1.2460940 (940)	total: 56.9s	remaining: 3.57s
941:	learn: 1.1773234	test: 1.2455683	best: 1.2455683 (941)	total: 57s	remaining: 3.51s
942:	learn: 1.1772128	test: 1.2454763	best: 1.2454763 (942)	total: 57.1s	remaining: 3.45s
943:	learn: 1.1768568	test: 1.2454508	best: 1.2454508 (943)	total: 57.1s	remaining: 3.39s
944:	learn: 1.1766807	test: 1.2453383	best: 1.2453383 (944)	total: 57.2s	remaining: 3.33s
945:	learn: 1.1765374	test: 1.2442561	best: 1.2442561 (945)	total: 57.2s	remaining: 3.27s
946:	learn: 

In [34]:
from sklearn.metrics import mean_squared_error

preds = model.predict(test_features)
RMSE = mean_squared_error(test_target,preds)**0.5
print(RMSE)

1.22704355853187


**RMSE가 1.2270**

## 6. 결과 및 결언

In [32]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [35]:
# CatBoost 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [37]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [38]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

REG_YYMM CARD_SIDO_NM           STD_CLSS_NM           AMT
id                                                           
0     202004           강원            건강보조식품 소매업  1.659240e+08
1     202004           강원               골프장 운영업  2.059787e+09
2     202004           강원           과실 및 채소 소매업  8.286259e+08
3     202004           강원     관광 민예품 및 선물용품 소매업  3.524188e+07
4     202004           강원  그외 기타 분류안된 오락관련 서비스업  9.251500e+06